<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/09_17_22_SBERT%2Bpt_biggraph_Embeddings_NN_Politifact_epochs400.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%%capture
!pip install -U sentence-transformers wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.0 MB/s 
     |████████████████████████████████| 1.8 MB 51.7 MB/s 
     |████████████████████████████████| 4.9 MB 48.2 MB/s 
     |████████████████████████████████| 1.3 MB 61.5 MB/s 
     |████████████████████████████████| 120 kB 66.8 MB/s 
     |████████████████████████████████| 6.6 MB 51.1 MB/s 
     |████████████████████████████████| 181 kB 46.2 MB/s 
     |████████████████████████████████| 158 kB 54.2 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 157 kB 52.9 MB/s 
     |████████████████████████████████| 157 kB 51.2 MB/s 
     |████████████████████████████████| 157 kB 56.5 MB/s 
     |████████████████████████████████| 157 kB 60.5 MB/s 
     |████████████████████████████████| 157 kB 55.6 MB/s 
     |████████████████████████████████| 157 kB 58.4 MB/s 
     |█████████████████████████████

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import re
import json

from sentence_transformers import SentenceTransformer 

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import wandb
from wandb.keras import WandbCallback
from IPython.display import clear_output

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return 'eval/' + score_func.__name__.split("_")[0]

# Load data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = Path("/content/drive/MyDrive/ResearchFND")
assert base_dir.exists()

Mounted at /content/drive


In [5]:
dataset_id = 'politifact'

In [6]:
import ast

converters = {"retweets":ast.literal_eval, "tweets":ast.literal_eval}
df = pd.read_csv(base_dir/f"{dataset_id}_agg.csv", converters=converters)
df.head()

,Unnamed: 0,title,text,tweets,retweets,label,url,tweet_ids,num_retweets,log_num_retweets,num_tweets,log_num_tweets
0,0,Actress Emma Stone ‘For the first time in his...,NaN,[],"[1020554564334964741, 1020817527046197248, 106...",fake,NaN,[],2911,7.976595,0,0.0
1,1,Breaking President Trump makes English the of...,NaN,[],[],fake,NaN,[],0,0.000000,0,0.0
2,2,Friendly Fire … Charlottesville Car attacker ...,NaN,[],"[3265439004, 3250621593, 3253922920, 326691851...",fake,NaN,[],24,3.218876,0,0.0
3,3,If You Are Using This Toothpaste… Throw It Aw...,MightyLiving Blog\n\nHelpful inspiration from ...,[],"[911971426571255810, 1036749614853103616, 1033...",fake,mightynest.com/learn/research-by-concern/dange...,[],2569,7.851661,0,0.0
4,4,"""Face the Nation"" transcripts, August 26, 2012...","""Face the Nation"" transcripts, August 26, 2012...",[],[],real,https://web.archive.org/web/20120827001956/htt...,[],0,0.000000,0,0.0


# Using the embeddings for title and text from previous experiments

In [7]:
title_embedding_file = base_dir/f"{dataset_id}_sbert_title_embeddings.npy"

if title_embedding_file.exists():
    title_embeddings = np.load(title_embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)
    titles = df.title.tolist()
    title_embeddings = model.encode(titles, show_progress_bar=True)
    np.save(title_embedding_file, title_embeddings)

In [8]:
text_embedding_file = base_dir/f"{dataset_id}_sbert_fulltext_embeddings.npy"

if text_embedding_file.exists():
    text_embeddings = np.load(text_embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)
    texts = (df.title + "\n" + df.text).tolist()
    text_embeddings = model.encode(texts, show_progress_bar=True)
    np.save(text_embedding_file, text_embeddings)

In [9]:
df["num_retweets"] = df.retweets.map(lambda x: len(set(x)))
df["log_num_retweets"] = np.log1p(df.num_retweets.to_numpy())
df["num_tweets"] = df.tweets.map(lambda x: len(set([t["id"] for t in x])))
df["log_num_tweets"] = np.log1p(df.num_tweets.to_numpy())

# Training - Saving the machine learning model specifications

In [10]:
import tensorflow as tf

In [11]:
def get_model(d_h=64, p_in:float=0.2, p_out:float=0.2):
    model = tf.keras.Sequential([
        tf.keras.layers.Dropout(p_in), # input dropout
        tf.keras.layers.Dense(d_h, activation='relu'),
        tf.keras.layers.Dropout(p_out),  
        tf.keras.layers.Dense(1)
    ])

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                  optimizer='adam',metrics=['accuracy'])
    return model

In [12]:
def train(fold_id, train_idx, test_idx, params):

    # setup
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    model = get_model()
    # training
    ckpt_filepath = f'ckpt/fold_{fold_id}'
    save_model_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=ckpt_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    history = model.fit(
        X_train, y_train, validation_data = (X_test, y_test), 
        batch_size = params['bs'] , epochs=params['epochs'],
        callbacks=[WandbCallback(), save_model_cb]
    )
    #evaluation
    model.load_weights(ckpt_filepath)
    logits = model.predict(X_test).squeeze()
    y_pred = (logits > 0.).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})
    return history

### PyTorch_BigGraph embeddings

In [13]:
edim = 128
bg_embeddings = np.load(base_dir/f'{dataset_id}_pt_biggraph_article_embeddings_{edim}.npy')
idx = np.load(base_dir/f"{dataset_id}_pt_biggraph_article_idx_{edim}.npy")

In [14]:
graph_embeddings = np.zeros((text_embeddings.shape[0], edim))
graph_embeddings[idx] = bg_embeddings

In [19]:
params = {
    "epochs":400,
    "bs":128,
    # 'lr':1e-3
}

In [20]:
X = np.concatenate([
    text_embeddings, 
    df.log_num_retweets.to_numpy()[..., None], 
    df.log_num_tweets.to_numpy()[..., None],
    graph_embeddings], axis=1)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((894, 898), (894,))

In [21]:
skf = StratifiedKFold(shuffle=True, random_state=124)

Creating a group and logging all train, test data in wandb for better tracking and visualisation.

In [22]:
GROUP = f"{dataset_id}-fulltext-sbert+tw+ptbg128-mlp+e400"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="nofolds", group=GROUP, name=f"{GROUP}-{fold_id}") as run:
        train(fold_id, train_idx, test_idx, params)
    break

wandb: Currently logged in as: saloni (saloniteam). Use `wandb login --relogin` to force relogin


wandb: WARNING Unable to compute FLOPs for this model.


Epoch 1/400
1/6 [====>.........................] - ETA: 2s - loss: 0.7086 - accuracy: 0.4375

wandb: Adding directory to artifact (/content/wandb/run-20220917_094310-1ocj07j8/files/model-best)... Done. 0.1s


6/6 [==============================] - 2s 201ms/step - loss: 0.6887 - accuracy: 0.4615 - val_loss: 0.6562 - val_accuracy: 0.4637
Epoch 2/400
1/6 [====>.........................] - ETA: 0s - loss: 0.6558 - accuracy: 0.4141

...

wandb: Adding directory to artifact (/content/wandb/run-20220917_094310-1ocj07j8/files/model-best)... Done. 0.1s


6/6 [==============================] - 1s 173ms/step - loss: 0.0334 - accuracy: 0.9902 - val_loss: 0.1281 - val_accuracy: 0.9441
Epoch 200/400
6/6 [==============================] - 0s 11ms/step - loss: 0.0334 - accuracy: 0.9930 - val_loss: 0.1287 - val_accuracy: 0.9441
Epoch 201/400
6/6 [==============================] - 0s 11ms/step - loss: 0.0246 - accuracy: 0.9916 - val_loss: 0.1326 - val_accuracy: 0.9441
Epoch 202/400
6/6 [==============================] - 0s 9ms/step - loss: 0.0335 - accuracy: 0.9930 - val_loss: 0.1360 - val_accuracy: 0.9497
Epoch 203/400
6/6 [==============================] - 0s 12ms/step - loss: 0.0331 - accuracy: 0.9902 - val_loss: 0.1342 - val_accuracy: 0.9441
Epoch 204/400
6/6 [==============================] - 0s 9ms/step - loss: 0.0351 - accuracy: 0.9930 - val_loss: 0.1375 - val_accuracy: 0.9441
Epoch 205/400
6/6 [==============================] - 0s 8ms/step - loss: 0.0287 - accuracy: 0.9930 - val_loss: 0.1439 - val_accuracy: 0.9441
Epoch 206/400
6/6 [===

wandb: Adding directory to artifact (/content/wandb/run-20220917_094310-1ocj07j8/files/model-best)... Done. 0.1s


6/6 [==============================] - 1s 188ms/step - loss: 0.0277 - accuracy: 0.9930 - val_loss: 0.1273 - val_accuracy: 0.9441
Epoch 209/400
6/6 [==============================] - 0s 10ms/step - loss: 0.0285 - accuracy: 0.9944 - val_loss: 0.1316 - val_accuracy: 0.9497
Epoch 210/400
6/6 [==============================] - 0s 8ms/step - loss: 0.0265 - accuracy: 0.9944 - val_loss: 0.1388 - val_accuracy: 0.9497
Epoch 211/400
6/6 [==============================] - 0s 8ms/step - loss: 0.0359 - accuracy: 0.9888 - val_loss: 0.1334 - val_accuracy: 0.9497
Epoch 212/400
6/6 [==============================] - 0s 8ms/step - loss: 0.0287 - accuracy: 0.9916 - val_loss: 0.1359 - val_accuracy: 0.9441
Epoch 213/400
6/6 [==============================] - 0s 10ms/step - loss: 0.0290 - accuracy: 0.9930 - val_loss: 0.1359 - val_accuracy: 0.9441
Epoch 214/400
6/6 [==============================] - 0s 9ms/step - loss: 0.0237 - accuracy: 0.9930 - val_loss: 0.1342 - val_accuracy: 0.9497
Epoch 215/400
6/6 [====

accuracy,▁▅▆▆▇▇▇▇▇▇██████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
loss,█▅▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▆▇▆▇▇▇▇▇▇▇▆▆▇▇▇▇▇▇▇▇███████████████████
val_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.9986
best_epoch,207
